In [10]:
from pathlib import Path

from data_scripts import get_newest_data_paths
from mandel_model import make_mandel_setup
from mandel_solvers import make_mandel_solver_space

from solver_selector.simulation_runner import make_simulation_runner

path = Path().parent / "../2"
load_data_paths = []
load_data_paths += get_newest_data_paths(path / "poro_coldstart_s", n_newest=1)
load_data_paths += get_newest_data_paths(path / "poro_coldstart_m", n_newest=1)
load_data_paths += get_newest_data_paths('poro_coldstart_gp', n_newest=1)

# assert len(load_data_paths) == 3

solver_space = make_mandel_solver_space(l_factor="dynamic")
simulation = make_mandel_setup(model_size="large")

In [11]:
simulation_runner = make_simulation_runner(
    solver_space=solver_space,
    params={
        "load_statistics_paths": load_data_paths,
        "print_solver": True,
        # "predictor": "gaussian_process",
        # 'regressor': 'mlp',
    },
)

Selecting from 1 solvers.
0 gmres - splitting_fixed_stress [primary - direct, secondary - direct, l_factor=0, primary_variable, method]
Using epsilon-greedy exploration
Using regressor: gradient_boosting
Warm start using data:
/home/firedrake/workspace/porepy_workspace/solver_selector/examples/3/../2/performance/poro_coldstart_s_11.npy
/home/firedrake/workspace/porepy_workspace/solver_selector/examples/3/../2/performance/poro_coldstart_m_10.npy
/home/firedrake/workspace/porepy_workspace/solver_selector/examples/3/performance/poro_coldstart_gp_13.npy


In [12]:
import pandas as pd
import numpy as np

predictor = simulation_runner.solver_selector.predictors[0]
X = np.array(predictor.memory_contexts)
y = np.array(predictor.memory_rewards)

data = pd.DataFrame(X, columns=['ts', 'ms', 'cfl_mean', 'cfl_max', 'l_factor'])
data['target'] = y
data.describe()

,ts,ms,cfl_mean,cfl_max,l_factor,target
count,3.000000e+02,300.000000,300.000000,300.000000,300.000000,300.000000
mean,2.302585e+00,9.522425,4.763271,2.675042,0.554912,0.096044
std,4.448312e-16,0.552540,0.683754,0.074138,0.165667,0.798898
min,2.302585e+00,8.884333,4.250704,2.650625,0.000000,-1.174203
25%,2.302585e+00,8.884333,4.380030,2.658094,0.526316,-0.921008
50%,2.302585e+00,9.452816,4.566659,2.666993,0.578947,0.254097
75%,2.302585e+00,10.230126,4.896762,2.672971,0.631579,0.923850
max,2.302585e+00,10.230126,9.988645,3.467773,1.000000,1.090734


In [13]:
ts_mean = data.ts.mean()
# data_max = data.ms.max() + (data.ms.max() - data.ms.min())
data_max = data.ms.max()
ms = np.linspace(data.ms.min(), data_max, 100)
cfl_mean = data.cfl_mean.mean()
cfl_max = data.cfl_max.mean()
l_factor = np.linspace(data.l_factor.min(), data.l_factor.max(), 100)

ms, l_factor = np.meshgrid(ms, l_factor, indexing='ij')

data1 = np.zeros((5, 100, 100))
data1[0] = ts_mean
data1[1] = ms
data1[2] = cfl_mean
data1[3] = cfl_max
data1[4] = l_factor
data1 = data1.reshape(5, -1)
data1 = data1.T

target1 = predictor.regressor.predict(data1)
df1 = pd.DataFrame(data1, columns=['ts', 'ms', 'cfl_mean', 'cfl_max', 'l_factor'])
df1['target'] = target1

In [14]:
# mean, std = predictor.regressor.predict(data1, return_std=True)
# df1['target'] = mean + std

In [15]:
from dash_app import make_app

app = make_app([data, df1], ['ms', 'l_factor', 'target'])

In [17]:
import torch
import gpytorch


class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(ard_num_dims=5)
            + gpytorch.kernels.LinearKernel(active_dims=[1])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


# initialize likelihood and model
predictor = simulation_runner.solver_selector.predictors[0]
X = torch.Tensor(predictor.memory_contexts)
y = torch.Tensor(predictor.memory_rewards)

likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(X, y, likelihood)

In [18]:
# this is for running the notebook in our testing framework
training_iter = 100


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(X)
    # Calc loss and backprop gradients
    loss = -mll(output, y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        # model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

/home/firedrake/firedrake/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning:

A not p.d., added jitter of 1.0e-06 to the diagonal

/home/firedrake/firedrake/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning:

A not p.d., added jitter of 1.0e-05 to the diagonal



Iter 1/100 - Loss: 0.817   noise: 0.693
Iter 2/100 - Loss: 0.778   noise: 0.644
Iter 3/100 - Loss: 0.739   noise: 0.598
Iter 4/100 - Loss: 0.700   noise: 0.554
Iter 5/100 - Loss: 0.660   noise: 0.513
Iter 6/100 - Loss: 0.620   noise: 0.474
Iter 7/100 - Loss: 0.579   noise: 0.437
Iter 8/100 - Loss: 0.537   noise: 0.403
Iter 9/100 - Loss: 0.495   noise: 0.370
Iter 10/100 - Loss: 0.452   noise: 0.340
Iter 11/100 - Loss: 0.409   noise: 0.312
Iter 12/100 - Loss: 0.366   noise: 0.286
Iter 13/100 - Loss: 0.321   noise: 0.261
Iter 14/100 - Loss: 0.277   noise: 0.239
Iter 15/100 - Loss: 0.232   noise: 0.218
Iter 16/100 - Loss: 0.186   noise: 0.198
Iter 17/100 - Loss: 0.140   noise: 0.181
Iter 18/100 - Loss: 0.094   noise: 0.164
Iter 19/100 - Loss: 0.048   noise: 0.149
Iter 20/100 - Loss: 0.001   noise: 0.136
Iter 21/100 - Loss: -0.046   noise: 0.123
Iter 22/100 - Loss: -0.093   noise: 0.112
Iter 23/100 - Loss: -0.140   noise: 0.101
Iter 24/100 - Loss: -0.188   noise: 0.092
Iter 25/100 - Loss: -

/home/firedrake/firedrake/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning:

A not p.d., added jitter of 1.0e-04 to the diagonal



NotPSDError: Matrix not positive definite after repeatedly adding jitter up to 1.0e-04.